In [1]:
import json
import pandas as pd

In [2]:
METADATA_CSV = "metadata.csv"
GENOMIC_NUCLEOTIDE_CSV = "genomic.csv"
PROTEIN_CSV = "protein.csv"

In [3]:
df = pd.read_csv(METADATA_CSV)
dfg = pd.read_csv(GENOMIC_NUCLEOTIDE_CSV)
dfp = pd.read_csv(PROTEIN_CSV)

In [4]:
print(f"Metadata shape {df.shape}")

Metadata shape (41679, 7)


In [5]:
print(f"Genomic shape {dfg.shape}")

Genomic shape (41762, 3)


In [6]:
print(f"Protein shape {dfg.shape}")

Protein shape (41762, 3)


In [7]:
# join genomic data to metadata
df = df.join(dfg.set_index('accession'), on='accession')

In [8]:
# join protein data to metadata
df = df.join(dfp.set_index('accession'), on='protein_accession')
df.rename(columns={"protein":"sgene_protein", "protein_desc":"sgene_protein_desc"}, inplace=True)

In [9]:
df.dropna(inplace=True)

In [10]:
def cut_sgene(row):
    begin = int(row['sgene_begin']) - 1
    end = int(row['sgene_end'])
    sgene = row['genome'][begin:end]
    return sgene

with pd.option_context('mode.chained_assignment', None):
    df['sgene_nucleotide'] = df.apply(cut_sgene, axis=1)

In [11]:
df

,accession,protein_accession,collection_date,sgene_begin,sgene_end,location,region,genome,genome_desc,sgene_protein,sgene_protein_desc,sgene_nucleotide
0,MN908947.3,QHD43416.1,2019-12,21563,25384,China,Asia,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...,Severe acute respiratory syndrome coronavirus ...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,1-1273 surface glycoprotein [organism=Severe a...,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
1,NC_045512.2,YP_009724390.1,2019-12,21563,25384,China,Asia,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...,Severe acute respiratory syndrome coronavirus ...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,1-1273 surface glycoprotein [organism=Severe a...,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
2,MN985325.1,QHO60594.1,2020-01-19,21563,25384,USA,North America,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...,Severe acute respiratory syndrome coronavirus ...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,1-1273 surface glycoprotein [organism=Severe a...,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
3,MN975262.1,QHN73810.1,2020-01-11,21563,25384,China,Asia,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...,Severe acute respiratory syndrome coronavirus ...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,1-1273 surface glycoprotein [organism=Severe a...,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
4,MN938384.1,QHN73795.1,2020-01-10,21531,25352,China: Shenzhen,Asia,CAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTT...,Severe acute respiratory syndrome coronavirus ...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,1-1273 surface glycoprotein [organism=Severe a...,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
...,...,...,...,...,...,...,...,...,...,...,...,...
41674,MW617535.1,QRU91038.1,2021-01-31,21532,25353,USA: Georgia,North America,CCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTT...,Severe acute respiratory syndrome coronavirus ...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,1-1273 surface glycoprotein [organism=Severe a...,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
41675,MW617534.1,QRU91026.1,2021-01-30,21535,25356,USA: Louisiana,North America,AAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAA...,Severe acute respiratory syndrome coronavirus ...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,1-1273 surface glycoprotein [organism=Severe a...,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
41676,MW617533.1,QRU91014.1,2021-01-31,21532,25353,USA: California,North America,CCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTT...,Severe acute respiratory syndrome coronavirus ...,MFVFFVLLPLVSIQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,1-1273 surface glycoprotein [organism=Severe a...,ATGTTTGTTTTTTTTGTTTTATTGCCACTAGTCTCTATTCAGTGTG...
41677,MW617516.1,QRU90810.1,2021-02-02,21558,25379,USA: California,North America,AGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCT...,Severe acute respiratory syndrome coronavirus ...,MFVFLVLLPLVSIQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,1-1273 surface glycoprotein [organism=Severe a...,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTATTCAGTGTG...


In [12]:
df.to_csv("ncbi_sgene_nuc_pro_full.csv", index=False)

In [24]:
with open("ncbi_sgene_nuc_full.fasta", "w") as fasta_file:
    for _, row in df.iterrows():
        fasta_file.write(f">{row['accession']}\n")
        fasta_file.write(f"{row['sgene_nucleotide']}\n")

In [25]:
letters = ['R', 'M', "S", "B", "H", "N", "Y", "K", "W", "D", "V"]
df_good = df[~df['sgene_nucleotide'].str.contains("|".join(letters))]

with open("ncbi_sgene_nuc_good.fasta", "w") as fasta_file:
    for _, row in df_good.iterrows():
        fasta_file.write(f">{row['accession']}\n")
        fasta_file.write(f"{row['sgene_nucleotide']}\n")

In [33]:
df_good

,accession,protein_accession,collection_date,sgene_begin,sgene_end,location,region,genome,genome_desc,sgene_protein,sgene_protein_desc,sgene_nucleotide
0,MN908947.3,QHD43416.1,2019-12,21563,25384,China,Asia,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...,Severe acute respiratory syndrome coronavirus ...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,1-1273 surface glycoprotein [organism=Severe a...,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
1,NC_045512.2,YP_009724390.1,2019-12,21563,25384,China,Asia,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...,Severe acute respiratory syndrome coronavirus ...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,1-1273 surface glycoprotein [organism=Severe a...,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
2,MN985325.1,QHO60594.1,2020-01-19,21563,25384,USA,North America,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...,Severe acute respiratory syndrome coronavirus ...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,1-1273 surface glycoprotein [organism=Severe a...,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
3,MN975262.1,QHN73810.1,2020-01-11,21563,25384,China,Asia,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...,Severe acute respiratory syndrome coronavirus ...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,1-1273 surface glycoprotein [organism=Severe a...,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
4,MN938384.1,QHN73795.1,2020-01-10,21531,25352,China: Shenzhen,Asia,CAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTT...,Severe acute respiratory syndrome coronavirus ...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,1-1273 surface glycoprotein [organism=Severe a...,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
...,...,...,...,...,...,...,...,...,...,...,...,...
41674,MW617535.1,QRU91038.1,2021-01-31,21532,25353,USA: Georgia,North America,CCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTT...,Severe acute respiratory syndrome coronavirus ...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,1-1273 surface glycoprotein [organism=Severe a...,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
41675,MW617534.1,QRU91026.1,2021-01-30,21535,25356,USA: Louisiana,North America,AAACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAA...,Severe acute respiratory syndrome coronavirus ...,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,1-1273 surface glycoprotein [organism=Severe a...,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTAGTCAGTGTG...
41676,MW617533.1,QRU91014.1,2021-01-31,21532,25353,USA: California,North America,CCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTT...,Severe acute respiratory syndrome coronavirus ...,MFVFFVLLPLVSIQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,1-1273 surface glycoprotein [organism=Severe a...,ATGTTTGTTTTTTTTGTTTTATTGCCACTAGTCTCTATTCAGTGTG...
41677,MW617516.1,QRU90810.1,2021-02-02,21558,25379,USA: California,North America,AGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCT...,Severe acute respiratory syndrome coronavirus ...,MFVFLVLLPLVSIQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...,1-1273 surface glycoprotein [organism=Severe a...,ATGTTTGTTTTTCTTGTTTTATTGCCACTAGTCTCTATTCAGTGTG...
